In [134]:
from IPython.display import display
import ipywidgets as widgets
from IPython.display import clear_output
import numpy as np
from ipywidgets import Video, Image
from glob import glob
import os
import time
import cv2
import time
import matplotlib.pyplot as plt
import json
import subprocess, shlex
import PIL as pil
import random
import functools

In [255]:
def find_frame_at_given_time(frame_num, video_path):
    
    
    vid_cap           = cv2.VideoCapture(video_path)
    fps               = vid_cap.get(cv2.CAP_PROP_FPS)
    total_frame       = int(vid_cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_num         = max(0, frame_num)
    
    vid_cap.set(1, frame_num)
    _, frame = vid_cap.read()
    
    
    return frame


def read_json(json_path):
    with open(json_path, 'rb') as f:
        return json.load(f)
    
def add_ratio_button(display_color, display_pattern):
    gender = widgets.RadioButtons(
        options=['good', 'bad'],
        value=None,
        layout={'width': 'max-content'}, 
        description='Gender:',
        disabled=False
    )

        
    category = widgets.RadioButtons(
        options=['good', 'bad'],
        value=None, 
        layout={'width': 'max-content'},
        description='Category:',
        disabled=False
    )

    if display_color:
        color = widgets.RadioButtons(
            options=['good', 'bad'],
            value=None, 
            layout={'width': 'max-content'}, 
            description='Color :',
            disabled=False
        )
    else:
        color = widgets.RadioButtons(
            options=['good', 'bad', 'NO'],
            value='NO', 
            layout={'width': 'max-content'}, 
            description='Color :',
            disabled=False
        )
        

    if display_pattern:
        pattern = widgets.RadioButtons(
            options=['good', 'bad'],
            value=None,
            layout={'width': 'max-content'},
            description='Pattern:',
            disabled=False
        )   
        
    else:
        pattern = widgets.RadioButtons(
            options=['good', 'bad', 'NO'],
            value='NO',
            layout={'width': 'max-content'},
            description='Pattern:',
            disabled=False
        )   

    frame = widgets.RadioButtons(
        options=['good', 'bad'],
        value=None,
        layout={'width': 'max-content'},
        description='Frame:',
        disabled=False
    ) 
        
    return frame, gender, category, color, pattern

def analysis(json_path, result_path, qa_status_path): #image array    
    hold = False
    
    def get_qa_status(qa_status_path): #get paths from qa_status.txt
        with open(qa_status_path, 'r') as f:
            size = os.path.getsize(qa_status_path)
            if size != 0:
                return  [i.replace('\n', '') for i in f.readlines()]

            else:
                return []

    def next_item(): 
        clear_output()
        analysis(new_json, result_path, qa_status_path)
        
        
    def click_submit(arg, argument):

        frame, gender, category, color, pattern, object_name = arg
        
        if gender.value and color.value and category.value and pattern.value and frame.value is not None:
            with open(result_path, 'a') as f:
                f.write(f"{object_name},{frame.value},{gender.value},{category.value},{color.value},{pattern.value}\n")
            print("recorded")
            argument.disabled = True
            
        else:
            print('please fill up all section')  
            
            
    def Done(passed_arg, arg):
        with open(qa_status_path, 'w') as f:
            f.write('\n'.join(passed_arg))
        next_item()
        
            
    def display_adds(item, object_name, series_name, video_path, display_color, display_pattern):
        global new_json, result_path
        #radio buttons
        frame, gender, category, color, pattern = add_ratio_button(display_color, display_pattern)

       

        frame_num  = item['features']['maxConfFrameNumber']
        
        if display_color and display_pattern:
            img_path   = f'{images_root_path}/{series_name}/multi_color_image_matching/{object_name}/*.png'
            img_path   = glob(img_path)[0]
        
        elif display_color and not(display_pattern):
            img_path   = f'{images_root_path}/{series_name}/multi_color/{object_name}/*.png'
            img_path   = random.shuffle(glob(img_path))[0]
            
        elif not(display_color):
            img_path   = f'{detected_path}/{object_name}'
                        
        final_image  = cv2.resize(cv2.imread(img_path), (300, 300))
        frame_       = cv2.resize(find_frame_at_given_time(frame_num, video_path), (300, 300))
        
        concated     = cv2.hconcat([frame_, final_image])
        cv2.imwrite('temp_file.png', concated)
        video       = Image.from_file('temp_file.png',  width = 600, height=600)
        
        #submit button
        arg    = frame, gender, category, color, pattern, object_name
        submit = widgets.Button(description = 'Submit', disabled=False)
        submit.on_click(functools.partial(click_submit, arg))
        
        #display 
        print("--> "+ img_path)
        content = widgets.HBox([video])
        if display_color and display_pattern:
            radio = widgets.HBox([frame, gender, category, color, pattern])
            
        elif display_color and not(display_pattern):
            radio = widgets.HBox([frame, gender, category, color])
            
        elif not(display_color):
            radio = widgets.HBox([frame, gender, category])
            
        display(widgets.VBox([content, radio, submit, widgets.Output()]))
        
        
        

    start          = time.time()
    qa_completed   = get_qa_status(qa_status_path)  
    new_json       = list(set(json_path) - set(qa_completed))
    
    #If there is no json_left to evaluate task is finished
    if len(new_json) == 0:
        print("CONGRATS...TASK FINITO !!")
        return 0
    
    
    rand_seed  = np.random.choice(range(0, 10))
    np.random.seed(rand_seed)
    rand_index = np.random.choice(range(0, len(new_json)))
    
    selected_json_path  = new_json[rand_index]
    selected_json       = read_json(selected_json_path)
    
    
    for items in selected_json['analyticsInfo']:
        hold        = True
        object_name = items['objectImage']
        series_name = '_'.join(object_name.split('_')[0:2])
        vid_path    = f"{video_root}/{series_name}/{'_'.join(object_name.split('_')[0:3])}.mp4"
        if items['recommended_ads'] != {}:
            display_adds(items, object_name, series_name, vid_path, display_color=True, display_pattern=True)
            
        else:
            obj_path    = f'{images_root_path}/series_name/multi_color/{object_name}'
            if os.path.isfile(obj_path):
                display_adds(items, object_name, series_name, vid_path, display_color=True, display_pattern=False)
                
            else:
                display_adds(items, object_name, series_name, vid_path, display_color=False, display_pattern=False)
           
    qa_completed  = qa_completed + [selected_json_path]
    done = widgets.Button(description = 'Done')  
    done.on_click(functools.partial(Done, qa_completed))
    output = widgets.Output()
    display(widgets.VBox([done]), output)

In [256]:
version          = 'frame_selection'

video_root       = '/notebooks/common-dataset/common/test_mx/video_segmented'
images_root_path = f'/notebooks/common-dataset/common/test_mx/{version}/Suggested_ads'
result_path      = f'/notebooks/common-dataset/common/test_mx/{version}/result.txt'
json_file_root   = '/notebooks/common-dataset/common/test_mx/json_frame_selection'
detected_path    = '/notebooks/static-clickable-poster/analytics-app/app-code/temp-media-folder/analytics'
QA_complted_list = f'/notebooks/common-dataset/common/test_mx/{version}/qa_complted.txt'

json_file_paths  = glob(f'{json_file_root}/*.json')



In [257]:
len(json_file_paths)

93

In [258]:
with open(QA_complted_list, 'a') as f:
    pass

analysis(json_file_paths, result_path, QA_complted_list)

['/notebooks/common-dataset/common/test_mx/json_frame_selection/BasementCompany_S1Ep1_001.json', '/notebooks/common-dataset/common/test_mx/json_frame_selection/Aashram_S1Ep1_024.json', '/notebooks/common-dataset/common/test_mx/json_frame_selection/EkThiBegum_E01_013.json', '/notebooks/common-dataset/common/test_mx/json_frame_selection/Ray_S1Ep1_010.json', '/notebooks/common-dataset/common/test_mx/json_frame_selection/Ray_S1Ep1_016.json', '/notebooks/common-dataset/common/test_mx/json_frame_selection/TheInsidersS1Ep2_high_008.json', '/notebooks/common-dataset/common/test_mx/json_frame_selection/Aashram_S1Ep1_010.json', '/notebooks/common-dataset/common/test_mx/json_frame_selection/Bhaukaal_S2Ep2_000.json', '/notebooks/common-dataset/common/test_mx/json_frame_selection/Bhaukaal_S2Ep2_007.json', '/notebooks/common-dataset/common/test_mx/json_frame_selection/Ray_S1Ep1_020.json', '/notebooks/common-dataset/common/test_mx/json_frame_selection/EkThiBegum_E02_012.json', '/notebooks/common-data

--> /notebooks/static-clickable-poster/analytics-app/app-code/temp-media-folder/analytics/EkThiBegum_E01_006_2_F_H2S12V2_half_tshirt.jpg


Output()

In [147]:
dir(Image)

['__class__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_notifiers',
 '_call_widget_constructed',
 '_comm_changed',
 '_compare',
 '_default_keys',
 '_display_callbacks',
 '_dom_classes',
 '_from_file',
 '_gen_repr_from_keys',
 '_get_embed_state',
 '_get_repr',
 '_guess_format',
 '_handle_custom_msg',
 '_handle_displayed',
 '_handle_msg',
 '_holding_sync',
 '_ipython_display_',
 '_is_numpy',
 '_load_file_value',
 '_lock_property',
 '_log_default',
 '_model_module',
 '_model_module_version',
 '_model_name',
 '_msg_callbacks',
 '_notify_trait',
 '_property_lock',
 '_register_validator',
 '_remove_notifiers',
 '_repr_keys',
 '_sen

In [215]:
def done_(arg):
    
    arg.disabled=True
    
done = widgets.Button(description = 'Done')  
done.on_click(done_)
output = widgets.Output()
display(widgets.VBox([done]), output)

Output()

In [209]:
dir(submit.disabled)

['__abs__',
 '__add__',
 '__and__',
 '__bool__',
 '__ceil__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__divmod__',
 '__doc__',
 '__eq__',
 '__float__',
 '__floor__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getnewargs__',
 '__gt__',
 '__hash__',
 '__index__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__le__',
 '__lshift__',
 '__lt__',
 '__mod__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__or__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__rdivmod__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rfloordiv__',
 '__rlshift__',
 '__rmod__',
 '__rmul__',
 '__ror__',
 '__round__',
 '__rpow__',
 '__rrshift__',
 '__rshift__',
 '__rsub__',
 '__rtruediv__',
 '__rxor__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__truediv__',
 '__trunc__',
 '__xor__',
 'bit_length',
 'conjugate',
 'denominator',
 'from_bytes',
 'imag',
 'numerator',
 'real',
 'to_bytes']